In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
hrs22_path = '/home/csi20local/Documents/HRS22/'

In [ ]:
def CellNumberProportionAkinetic(path, threshold):
    
    df_strains = pd.read_csv(path)
    
    NumTotCells = df_strains.shape[0]
    NumAkinCells = df_strains[abs(df_strains['Area']) < threshold].shape[0]
    
    ans = NumAkinCells / NumTotCells
    
    return ans

In [ ]:
def AreaProportionAkinetic(path, threshold):
    
    SW_BE_path = os.path.dirname(path)
    
    df_strains = pd.read_csv(path)
    df_areas = pd.read_csv(f'{SW_BE_path}/transformed-4-clip-areas.txt', delim_whitespace=True, skiprows=1,
                           names = ['Cell', 'Cell Number', 'Area_label', 'Area'])
    
    area_tot = sum(df_areas['Area'])
    df_areas_akin = df_areas[abs(df_strains['Area']) < threshold]
    area_akin = sum(df_areas_akin['Area'])

    
    ans = area_akin / area_tot
    
    return ans

In [ ]:
FIMH_Cases = ('01', '02', '05', '06', '07', '08', '09', '12', '15', '16')
All_Cases = ('01', '02', '05', '06', '07', '08', '09', '10', '12', '14', '15', '16',
             '17', '18', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '32')
All_Cases = ['CT-CRT-' + Case for Case in All_Cases]
    
Add = ['Normal-1', 'Normal-3', 'EBR-01', 'EBR-02']
All_Cases = All_Cases + Add

TDownsampled_Cases = ['CT-CRT-' + x for x in ('21', '23', '24', '25', '26', '27', '28', '29', '30', '32')]
TDownsampled_Cases.append('EBR-01')
TDownsampled_Cases.append('EBR-02')

AF_Cases = ['CT-CRT-' + x for x in ('10', '19', '20', '23', '25', '26')]
AF_Cases.append('EBR-01')
AF_Cases.append('EBR-02')

Normal_SR_Only_Cases = ['CT-CRT-' + x for x in ('05', '17', '28', '32')]
Normals = ['Normal-1', 'Normal-3']
Normal_SR_Only_Cases = Normal_SR_Only_Cases + Normals

### SET THRESHOLD FOR AKINETIC CELL IN CELL BELOW THIS

In [ ]:
Dict = {}

for Case in All_Cases:
    
    thresh = 0.05
    
    if Case in TDownsampled_Cases:
        proportion = AreaProportionAkinetic(f'/home/csi20/Dropbox/phd/Data/RG_CT_Cases/{Case}/MT-HiRes-TDownsampled/SW-0.0-BE-4e-9/area-strains-4.csv',
                                       thresh)
    
    else:
        proportion = AreaProportionAkinetic(f'/home/csi20/Dropbox/phd/Data/RG_CT_Cases/{Case}/MT-HiRes/SW-0.0-BE-4e-9/area-strains-4.csv',
                                       thresh)
        
    Dict[Case] = proportion
                
#     print(Case, proportion)

In [ ]:
AreaProportionAkinetic(path='/home/csi20/Dropbox/phd/Data/RG_CT_Cases/CT-CRT-17/MT-HiRes/SW-0.0-BE-4e-9/area-strains-4.csv', threshold=0.05)

In [ ]:
AF_data = []
HF_data = []

for Case in All_Cases:
    
    if Case in AF_Cases:
        AF_data.append(np.array(Dict[Case]))
        
    else:
        HF_data.append(np.array(Dict[Case]))


In [ ]:
Normal_SR_Only_data = []

for Case in All_Cases:
    
    if Case in Normal_SR_Only_Cases:
        Normal_SR_Only_data.append(np.array(Dict[Case]))

In [ ]:
all_data = HF_data + AF_data

In [ ]:
HF_cols = ['tab:blue'] * 22
AF_cols = ['tab:orange'] * 8
cols = HF_cols + AF_cols

In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid")
sns.barplot(data=all_data, palette=cols)

# plt.title(f'Threshold for Akinesia = {thresh}')
ax = plt.gca()
y_max = 0.3
ax.set_ylim(0.0, y_max)
ax.set_ylabel('Akinetic fraction of LA surface', fontsize=17)
ax.set_xlabel('Cases', fontsize=17)

ax.set_xticks([10.5, 25.5])
ax.set_xticklabels(['HF Only', 'HF and AF'], fontsize=14)

ax.set_yticks(np.arange(0.0, 0.35, 0.05))
ax.set_yticklabels([0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30], fontsize=14)

ax.vlines(x = 21.5, ymin = 0, ymax = 0.6, color = 'black', linewidth=3, ls='--')

# Calc mean and std for each patient group
mean_AF = round(np.mean(AF_data), 3)
std_AF = round(np.std(AF_data), 3)

mean_HF = round(np.mean(HF_data), 3)
std_HF = round(np.std(HF_data), 3)

# Text for Mean and std for each patient group
ax.text(7, y_max-0.020, f'$\mu =$ {mean_HF}', fontsize=14)
ax.text(7, y_max-0.040, f'$\sigma =$ {std_HF}0', fontsize=14)

ax.text(23, y_max-0.020, f'$\mu =$ {mean_AF}', fontsize=14)
ax.text(23, y_max-0.040, f'$\sigma =$ {std_AF}', fontsize=14)

# ax.xaxis.set_major_locator(plt.NullLocator())

# ax1.set_ylim(0.0, 0.3)

# plt.savefig(hrs22_path + f'barplot_thresh_{thresh}-correct-bf-notitle.png', bbox_inches='tight')

In [ ]:
## Visualising Healthy/SR Only cases and HF + AF Cases

Healthy_AF_data = Normal_SR_Only_data + AF_data

import seaborn as sns

sns.barplot(data=Healthy_AF_data)

plt.title(f'Thresh {thresh}')


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
true_lab = []
for i in range(0, 22):
    true_lab.append(0)
    
for i in range(0, 8):
    true_lab.append(1)

In [ ]:
AUC = roc_auc_score(y_true = true_lab, y_score=all_data)
AUC = round(AUC, 2)
AUC

In [ ]:
scaled_data = [data / 0.2 for data in all_data]

roc_auc_score(y_true = true_lab, y_score=scaled_data)

In [ ]:
fpr, tpr, _ = roc_curve(y_true=true_lab, y_score=all_data)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true=true_lab, y_score=all_data)

In [ ]:
ns_probs = [0 for _ in range(len(true_lab))]
# ns_probs

ns_fpr, ns_tpr, _ = roc_curve(true_lab, ns_probs)
ns_auc = roc_auc_score(true_lab, ns_probs)

In [ ]:
plt.plot(fpr, tpr, marker='.', color='tab:orange', lw=3)
plt.plot(ns_fpr, ns_tpr, linestyle='--', color='tab:blue', lw=3)

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="white", palette=None)
sns.set_theme(style="ticks", rc=custom_params)

ax = plt.gca()
ax.set_ylim(-0.02, 1.05)
ax.set_xlim(-0.02, 1.05)

# ax.set_title(f'ROC curve using Threshold = {thresh}')
ax.set_ylabel('Sensitivity', fontsize=20)
ax.set_xlabel('1 - Specificity', fontsize=20)

ax.set_xticks(np.arange(0.0, 1.2, 0.2))
ax.set_yticks(np.arange(0.0, 1.2, 0.2))

ax.set_xticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=16)
ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=16)

ax.text(0.65, 0.3, f'AUC = {AUC}', fontsize=23)

# plt.savefig(hrs22_path + f'roccurve_thresh_{thresh}-correct-bf-notitle-bigf.png', bbox_inches='tight')

In [ ]:
plt.plot(fpr, tpr, marker='.', color='tab:orange', lw=3)
plt.plot(ns_fpr, ns_tpr, linestyle='--', color='tab:blue', lw=3)

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="white", palette=None)
sns.set_theme(style="ticks", rc=custom_params)

ax = plt.gca()
ax.set_ylim(-0.02, 1.05)
ax.set_xlim(-0.02, 1.05)

# ax.set_title(f'ROC curve using Threshold = {thresh}')
ax.set_ylabel('Sensitivity', fontsize=20)
ax.set_xlabel('1 - Specificity', fontsize=20)

ax.set_xticks(np.arange(0.0, 1.2, 0.2))
ax.set_yticks(np.arange(0.0, 1.2, 0.2))

ax.set_xticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=16)
ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=16)

ax.text(0.65, 0.3, f'AUC = 0.83', fontsize=23)

# plt.savefig("/home/csi20/Dropbox/phd/Documents/HRS22/roccurve_thresh_{thresh}-correct-bf-notitle-bigf-dpi300.png", bbox_inches='tight', dpi=300)

In [ ]:
tpr[6] # Sensitivity

In [ ]:
1 - fpr[6] # Specificity

In [ ]:
thresholds[6]

In [ ]:
sns.set_style('whitegrid')

boxplt_cols = ['tab:blue', 'tab:orange']

sns.boxplot(data=[HF_data, AF_data], palette=boxplt_cols)

ax = plt.gca()
ax.set_ylim(0.0, 0.29)

ax.set_xticks([0,1])
ax.set_xticklabels(['HF', 'HF + AF'], fontsize=20)

ax.set_yticks([0.0, 0.05, 0.10, 0.15, 0.20, 0.25])
ax.set_yticklabels([0, 5, 10, 15, 20, 25], fontsize=13)

ax.set_ylabel('Akinetic proportion\nof surface area [%]', fontsize=20)

ax.text(0.5, 0.265, "p value = 0.002", ha='center', va='bottom', fontsize=16)
ax.plot([0, 0, 1, 1], [0.255, 0.265, 0.265, 0.255], lw=1.2, c='black')

# plt.savefig(hrs22_path + f'boxplot_thresh_{thresh}.png', bbox_inches='tight')

In [ ]:
sns.set_style('whitegrid')

boxplt_cols = ['tab:blue', 'tab:orange']

sns.boxplot(data=[HF_data, AF_data], palette=boxplt_cols)

ax = plt.gca()
ax.set_ylim(0.0, 0.33)

ax.set_xticks([0,1])
ax.set_xticklabels(['HF Only', 'HF + AF'], fontsize=22)

ax.set_yticks([0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30])
ax.set_yticklabels([0, 5, 10, 15, 20, 25, 30], fontsize=15)

ax.set_ylabel('Akinetic proportion\nof LA surface area [%]', fontsize=22)

ax.text(0.5, 0.295, "p value = 0.002", ha='center', va='bottom', fontsize=16)
ax.plot([0, 0, 1, 1], [0.285, 0.295, 0.295, 0.285], lw=1.2, c='black')

ax.text(0.15, 0.245, "n = 22", ha='right', va='bottom', fontsize=16)
ax.text(0.87, 0.245, "n = 8", ha='left', va='bottom', fontsize=16)

# plt.savefig('/home/csi20/Dropbox/phd/Documents/HRS22/boxplot_thresh_{thresh}_dpi300.png', bbox_inches='tight', dpi=300)

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
ttest_ind(HF_data, AF_data)

## Mean, median, std of HF, AF + HF population and healthy population

In [ ]:
AF_mean = np.mean(AF_data)
HF_mean = np.mean(HF_data)

AF_std = np.std(AF_data)
HF_std = np.std(HF_data)

AF_med = np.median(AF_data)
HF_med = np.median(HF_data)

In [ ]:
print('AF mean: ', AF_mean)
print('HF mean: ', HF_mean)

In [ ]:
print('AF median: ', AF_med)
print('HF median: ', HF_med)

In [ ]:
print('AF std: ', AF_std)
print('HF std: ', HF_std)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

sns.barplot(ax=ax1, data=AF_data)
# ax1.set_ylim(0.0, 0.3)

ax2 = sns.barplot(ax=ax2, data=HF_data)
# ax.set_ylim(0.0, 0.3)

In [ ]:
for Case in All_Cases:
    
    thresh = 0.1
    
    if Case in TDownsampled_Cases:
        proportion = ProportionAkinetic(f'/home/csi20local/Data/RG_CT_Cases/{Case}/MT-HiRes-TDownsampled/SW-0.0-BE-4e-9/area-strains-4.csv',
                                       thresh)
    
    else:
        proportion = ProportionAkinetic(f'/home/csi20local/Data/RG_CT_Cases/{Case}/MT-HiRes/SW-0.0-BE-4e-9/area-strains-4.csv',
                                       thresh)
        
    print(Case, proportion)